In [ ]:
import numpy as np
import cv2
from scipy.stats import zscore
from sklearn.model_selection import StratifiedShuffleSplit

import mne
from mne import Epochs

from braindecode.datautil.preprocess import MNEPreproc, NumpyPreproc, preprocess
from braindecode.datautil.preprocess import exponential_moving_standardize
from braindecode.datasets.moabb import MOABBDataset

from tensorflow.keras.models import load_model
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import AveragePooling2D, Flatten,Conv2D, Dense, SpatialDropout2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#--------------------------------
# Set training and model parameters
#--------------------------------

kfolds = 5        # number of test folds
val_perc = 0.2    # percentage of training data used for validation
n_epochs = 500    # max number of epochs
batch_size = 64   # size of training batch
lr = 1e-3         # learning rate

verbose = 0       # regulate print statements
path = '/home/matthijspals' # main folder
parts = np.arange(1,3)#0) # list of participants

stacked = True    # stack images on top of each other
n_ch = 2          # number of channels to use (2: C3 and C4, 3: C3, Cz and C4)
tmax = 4          # end time after stimulus to include, in s
tmin = 0          # start time after stimulus to include, in s
fs = 250.         # sampling frequency
samples = int((tmax-tmin)*fs+1) #number of samples per trial
im_x = 28         # image width
im_y = 28         # image height

lfreq, hfreq= 8, 30 # low and high frequencies for filtering and wavelet transform
freq_spec = np.logspace(*np.log10([lfreq, hfreq])) # frequencies for wavelet transform
n_cycles = freq_spec/2 # number of cycles (controls width) of morlet wavelets
factor_new = 1e-3 # for moving normalization
init_block_size = samples #for moving normalization

vall_accs = []    # initialize list for all validation accuracies 
test_accs = []    # initialize list for all test accuracies 
m_vall_accs = []  # initialize list for mean validation accuracies 
m_test_accs = []  # initialize list for mean test accuracies 

In [ ]:
#--------------------------------
# Creates a Convolutional Neural Net
#--------------------------------

def CNN(input_shape, lr):
    """Return a CNN model, given input shape and learning rate"""

    input_layer = Input(input_shape)

    #Define model        
    layer = Conv2D(filters=16,
        kernel_size=(5, 5),
        strides=(2, 2),
        activation='relu',
        use_bias=False,
    )(input_layer)

    layer = SpatialDropout2D(0.5)(layer)

    layer = Conv2D(
        filters=32,
        kernel_size=(3, 3),
        activation='relu',
        use_bias=False,
    )(layer)

    layer = SpatialDropout2D(0.5)(layer)

    layer = AveragePooling2D()(layer)

    layer = Conv2D(
        filters=8,
        kernel_size=(3, 3),
        padding='same',
        activation='relu',
        use_bias=False,
    )(layer)

    layer = SpatialDropout2D(0.5)(layer)

    layer = Flatten()(layer)

    layer = Dense(units=2,
                   activation='softmax',
                   use_bias=True,
                   )(layer)

    model = Model(input_layer, layer)

    #define optimizer
    opt = Adam(learning_rate = lr)

    #compile model
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

    return model

In [ ]:

#--------------------------------
# Load and preprocess data
#--------------------------------

def load_and_preprocess(part, n_ch, lfreq, hfreq, factor_new, init_block_size, tmin, tmax, fs, 
                        freq_spec, n_cycles, stacked, im_x, im_y):
    """load and preprocess data using MOABB, BrainDecode and MNE, see above cell for parameters"""

    dataset = MOABBDataset(dataset_name="BNCI2014004", subject_ids=[part])

    
    if n_ch ==2:
        ch_names =['C3', 'C4']
    else:
        ch_names =['C3', 'Cz', 'C4']

    # preprocess using MNE and Braindecode
    
    preprocessors = [
        # keep only EEG sensors
        MNEPreproc(fn='pick_types', eeg=True, meg=False, stim=False),
        # convert from volt to microvolt, directly modifying the numpy array
        NumpyPreproc(fn=lambda x: x * 1e6),
        # bandpass filter
        MNEPreproc(fn='filter', l_freq=lfreq-4, h_freq=hfreq+4),
        # pick eeg channels
        MNEPreproc(fn='pick_channels', ch_names =ch_names),
        # exponential moving standardization
        NumpyPreproc(fn=exponential_moving_standardize, factor_new=factor_new,
            init_block_size=init_block_size),
    ]

    preprocess(dataset, preprocessors)

    # spectrogram creation
    
    label_list = []
    image_list = []
    
    #5 sessions per participant
    for exp in range(5):
        
        #load data and annotions
        raw = dataset.datasets[exp].raw
        events, _ = mne.events_from_annotations(raw)
        epochs = Epochs(raw, events, tmin=tmin, tmax=tmax, baseline=None)
        trials = len(events)
   
        #Use Morlet wavelets to create a time-frequency representation
        power = mne.time_frequency.tfr_morlet(epochs, n_cycles=n_cycles, freqs=freq_spec, 
                                          average=False, return_itc=False, verbose=False, use_fft=True)
        #normalize spectrograms
        img = []
        for trial in range(trials):
            img.append(zscore(power.data[trial,0:n_ch,:,:].reshape(n_ch*freq_spec.shape[0],-1)))
        img = np.array(img)
        
        # add spectrograms and associated labels to lists
        labels = epochs.events[:, -1] - 1
        label_list.append(labels)
        image_list.append(np.squeeze(img))

    # do image compression
    
    image_comp_list = []
    for img in image_list:
        
        if not stacked:
            im_y *= 2
        
        img_comp = np.zeros((img.shape[0],im_y,im_x))
        
        for i in range(np.shape(img)[0]):
            temp =  cv2.resize(img[i], dsize=(im_x, im_y), interpolation=cv2.INTER_CUBIC)
            img_comp[i]=temp
        img_comp = np.expand_dims(img_comp, axis = 3)
        
        if not stacked:
            y_new =int(im_y/n_ch)
            img=np.zeros((img.shape[0],im_x,y_new,n_ch))
            for i in range(n_ch):
                img[:,:,:,i]=img_comp[:,i:i+y_new,:,0]
            img_comp=img
        
        #set range to [0, 1] (as SNNtoolbox requires)
        img_comp -= np.min(img_comp)
        img_comp /=np.max(img_comp)
        
        image_comp_list.append(img_comp)
        
    return image_comp_list, label_list

In [ ]:
#--------------------------------
# Training
#--------------------------------

for part in parts:

    
    vall_acc = [] # initialize list for all validation accuracy of one participant
    test_acc = [] # initialize list for all test accuracy of one participant

    #load and preprocess data
    image_comp_list, label_list = load_and_preprocess(part, n_ch, lfreq, hfreq, factor_new, init_block_size, 
                                                      tmin, tmax, fs, freq_spec, n_cycles, stacked, im_x, im_y)

    
    kfold_indices = []
    
    #get kfold indices per session    
    for sess in range(len(image_comp_list)):
        
        #split data per classes (to make stratified folds)
        class1 = np.where(label_list[sess])[0]
        class2 = np.where(label_list[sess]==0)[0]
        
        #shuffle in place
        np.random.shuffle(class1)
        np.random.shuffle(class2)
        
        #split into required number of folds
        fold_ind = []
        for fold in range(kfolds):
            indices = np.concatenate([class1[fold::kfolds],class2[fold::kfolds]])
            np.random.shuffle(indices)
            fold_ind.append(indices)
        kfold_indices.append(fold_ind)
        
    print("created kfold indices")
    
    # train per fold
    
    for fold in range(kfolds):
        x_train_all = []  # train + val data
        y_train_all = []  # train + val labels
        x_test = []       # test data
        y_test = []       # test labels
        
        # loop through sessions (5 per part) to get test and (train+val)
        for sess in range(len(image_comp_list)):

            test_index = kfold_indices[sess][fold]
            
            x_test.append(image_comp_list[sess][test_index])
            y_test.append(label_list[sess][test_index])

            train_index = np.ones(len(image_comp_list[sess]), dtype=bool)
            train_index[test_index] = False

            x_train_all.append(image_comp_list[sess][train_index])
            y_train_all.append(label_list[sess][train_index])

        x_train = []  # train data
        y_train = []  # train labels
        x_val = []    # val data
        y_val = []    # val labels
        
        # again loop through sessions, to do a stratified split of (train+val) in train and val
        for sess in range(len(image_comp_list)):
            sss = StratifiedShuffleSplit(n_splits=1, test_size=val_perc)
            for train_index, test_index in sss.split(x_train_all[sess], y_train_all[sess]):
                x_train.append(x_train_all[sess][train_index])
                y_train.append(y_train_all[sess][train_index])
                x_val.append(x_train_all[sess][test_index])
                y_val.append(y_train_all[sess][test_index])


        # finally concatenate the data of each session for val, train and test
        x_train=np.concatenate(x_train)
        y_train=to_categorical(np.concatenate(y_train))
        x_val=np.concatenate(x_val)
        y_val=to_categorical(np.concatenate(y_val))
        x_test=np.concatenate(x_test)
        y_test=to_categorical(np.concatenate(y_test))

        # save the data for each fold
        np.savez_compressed(path +"/spin/models/subject_"+str(part)+"/fold"+str(fold)+"/x_norm.npz", x_test)
        np.savez_compressed(path + "/spin/models/subject_"+str(part)+"/fold"+str(fold)+"/x_test.npz", x_test)
        np.savez_compressed(path + "/spin/models/subject_"+str(part)+"/fold"+str(fold)+"/y_test.npz", y_test)

        # path for storing model
        model_path = path + "/spin/models/subject_"+str(part)+"/fold"+str(fold)+"/tsfold" + str(fold)+".h5"
        
        # store model that performs best on val
        callbacks = [ModelCheckpoint(
            filepath=model_path,
            monitor='val_accuracy',
            mode='max',
            save_best_only=True)]

        # do training
        
        #get model
        input_shape = x_train[0].shape
        model = CNN(input_shape, lr)

        # Generate a print
        if verbose:
            print('------------------------------------------------------------------------')
            print(f'Training for fold {fold} ...')

        # Fit data to model
        history = model.fit(x_train,  y_train,
                              batch_size=batch_size,
                              epochs=n_epochs,
                              validation_data=(x_val,y_val),
                              verbose=0,
                       callbacks=callbacks)

        print("part: " + str(part) + " fold: " + str(fold))
        print("best validation: " +str(max(history.history["val_accuracy"])))
        vall_acc.append(max(history.history["val_accuracy"]))
        
        #load best performing model on val and test on test
        best_val_mod = load_model(model_path)
        acc = best_val_mod.evaluate(x_test, y_test)
        test_acc.append(acc[1])
    
    print("accuracies, part: " + str(part))
    print("vall accs: " + str(vall_acc))
    print("mean vall acc: " + str(np.mean(vall_acc)))
    print("test acc: " + str(test_acc))
    print("mean test acc: " + str(np.mean(test_acc)))
    vall_accs.append(vall_acc)
    test_accs.append(test_acc)
    m_vall_accs.append(np.mean(vall_acc))
    m_test_accs.append(np.mean(test_acc))